In [2]:
%load_ext autoreload
%autoreload 2

import os 
import shutil
import numpy as n
from datetime import date
from matplotlib import pyplot as plt
import time
import napari

In [3]:
os.chdir('/home/ali/packages/s2p-lbm/')

from suite3d.job import Job
from suite3d import lbmio, utils, ui
from suite3d import tiff_utils as tfu
from suite3d import file_utils as flu
from suite3d import nbtools as nbui
from suite3d import extension as ext

/home/ali/anaconda3/envs/suite3d-gpu/lib/python3.8/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/home/ali/anaconda3/envs/suite3d-gpu/lib/python3.8/site-packages/paramiko/transport.py:258: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


In [4]:
subjects_dir =   '/mnt/zortex-subjects/'
expt_info = { 
    'subject':         'SS002',
    'date' :          '2024-08-12',
    'expnum' :         list(n.arange(1,22))}

tifs, si_params, exp_str = flu.find_exp(subjects_dir, **expt_info, verbose=False)

job_params = {    
    # number of channels recorded in the tiff file, typically 30
    'n_ch_tif' : 26,
    # number of planes in the deeper cavity, typically 15
    'cavity_size' : 13,
    # convert from the Scanimage channel ordering to deep-to-shallow ordering
    # in our case, ScanImage channel numbers are in temporal order. 
    'planes' : n.array([ 0,  2,  4,  6,  8, 10, 12, 14,
                        16, 18, 20, 22, 24, 1,  3, 5,  7,9,11, 13, 15,17,19,21]),
    # number of files to use for the initial pass
    'n_init_files' :   4,
    
    # number of pixels to fuse between the ROI strips
    # the auto-detection doesn't always work well, recommend 
    # manually tuning it to reduce the stitching artifacts
    'fuse_shift_override' : 6,
    
    # will try to automatically estimate crosstalk using 
    # the shallowest crosstalk_n_planes planes. if you want to override,
    # set override_crosstalk = float between 0 and 1
    'subtract_crosstalk' : True,
    
    # volume rate in acquisition
    'fs' : flu.get_si_params(tifs[0])['vol_rate'],
    
    # 3D GPU registration - fast! 
    # for now, 3D + GPU are well-tested, other options might have minor bugs
    # if you run into them, let us know! we'll fix them
    '3d_reg' : True,
    'nonrigid' : True,
    'gpu_reg' : True,
    'save_dtype' : 'float16',
}

In [5]:

job_params['fs'] = si_params['vol_rate']

savedir = os.path.join('/mnt/md0/runs', expt_info['subject'], expt_info['date'])
os.makedirs(savedir, exist_ok=True)

# Create the job
job = Job('/mnt/md0/runs',exp_str, tifs = tifs,
          params=job_params, create=False, overwrite=False, verbosity = 3)

   Found and loaded params from /mnt/md0/runs/s3d-SS002_2024-08-12_1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-17-18-19-20-21/params.npy


In [5]:
# optional parameters for initialization
# load 1 file to initialize, and select 200 frames randomly to do initalization on
job.params['n_init_files'] = 4
job.params['init_n_frames'] = None
job.params['init_file_sample_method'] = 'even'
job.params['voxel_size_um'] = (20, 2.5, 2.5)

In [6]:
# %%time
# summary = job.run_init_pass()

In [7]:
summary = job.load_summary()

In [8]:
img = summary['ref_img_3d']

In [9]:
# split the large tiffs into files of size 100 after registration
job.params['split_tif_size'] = 100

In [10]:
# %%time
# job.register()

In [11]:
# mov_full = job.get_registered_movie('registered_fused_data', 'fused')
# im_full = mov_full[:,:200].mean(axis=1).compute()

In [12]:
# tfu.show_tif(im_full[0])

In [13]:
# job.params['cell_filt_type'] = 'gaussian'

# job.params['npil_filt_xy_um'] = 70
# job.params['cell_filt_xy_um'] = 5
# job.params['sdnorm_exp'] = 0.80

# params_to_sweep = {
#     'cell_filt_xy_um' : (5,10),
#     'npil_filt_xy_um' : (40, 70, 100),
#     'sdnorm_exp' : (0.8,1.0),
# }
# job.sweep_corrmap(params_to_sweep, iter_limit=3)

In [14]:
# %%time
# corr_map = job.calculate_corr_map()

In [15]:
corr_map = job.load_corr_map_results()['vmap']
corr_map_thresh = ext.thresh_mask_corr_map(corr_map, thresh_window_size_pix = 51, corrmap_thresh_pct = 50)

In [16]:
# job.params['percentile'] : 99.9
# params_to_sweep = {
#     'activity_thresh' : [3.0, 5.0, 10.0, 20.0]
# }
# job.sweep_segmentation(params_to_sweep, vmap = corr_map_thresh, patches_to_segment=[15,], ts = (0,20000), all_combinations=True)

In [ ]:
# most important parameter - any value of the corrmap
# above this will be considered a peak for a possible ROI,
# and will be used as a "seed" to grow an ROI around it
# bigger number: fewer ROIs, only bright ones
# smaller number: many ROIs, increasingly worse quality
job.params['peak_thresh'] = 0.02

# optionally, bin the movie in time to speed up detection
# probably a good idea if you have high framerate (>5 Hz?)
job.params['detection_timebin'] = 1 

# when extending an ROI, compare its activity to its neighboring pixels
# in frames where the fluorescence is above this percentile
job.params['percentile'] : 99.9
job.params['activity_thresh'] = 5
job.params['extend_thresh'] = 0.015
job.params['max_iter'] = 5000
job.params['n_proc_detect'] = 16


job.segment_rois(vmap = corr_map_thresh)

      Found dir /mnt/md0/runs/s3d-SS002_2024-08-12_1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-17-18-19-20-21/segmentation
      Updating self.dirs tag segmentation
   Saved a copy of params at /mnt/md0/runs/s3d-SS002_2024-08-12_1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-17-18-19-20-21/segmentation
   Updated main params file
      Found dir /mnt/md0/runs/s3d-SS002_2024-08-12_1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-17-18-19-20-21/rois
      Updating self.dirs tag rois
   Saving results to /mnt/md0/runs/s3d-SS002_2024-08-12_1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-17-18-19-20-21/segmentation and /mnt/md0/runs/s3d-SS002_2024-08-12_1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-17-18-19-20-21/rois 
dict_keys(['max_img', 'mean_img', 'vmap', 'all_params'])
   Detecting from patch 1 / 88
      Found dir /mnt/md0/runs/s3d-SS002_2024-08-12_1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-17-18-19-20-21/segmentation/patch-0000
         Loading 34.42 GB movie to memory, shape: (53470, 24, 120, 120) 
         Loaded
         

/home/ali/packages/s2p-lbm/suite3d/extension.py:587: RuntimeWarning: invalid value encountered in divide
  frac_candidate_variance_on_roi = candidate_proj_on_roi / candidate_norms


         Added cell 49 at 07, 050, 035, peak: 0.168, thresh: 5.000, 6703 frames, 165 pixels
         Added cell 50 at 10, 056, 048, peak: 0.166, thresh: 5.000, 5946 frames, 161 pixels
         Added cell 51 at 05, 061, 056, peak: 0.163, thresh: 5.000, 6088 frames, 168 pixels
         Added cell 52 at 10, 037, 051, peak: 0.158, thresh: 5.000, 5749 frames, 157 pixels
         Added cell 53 at 09, 095, 067, peak: 0.157, thresh: 5.000, 5689 frames, 129 pixels
         Added cell 54 at 05, 076, 045, peak: 0.150, thresh: 5.000, 6113 frames, 120 pixels
         Added cell 55 at 13, 102, 064, peak: 0.143, thresh: 5.000, 6436 frames, 79 pixels
         Added cell 56 at 10, 101, 050, peak: 0.142, thresh: 5.000, 5975 frames, 84 pixels
         Added cell 57 at 11, 083, 051, peak: 0.137, thresh: 5.000, 5432 frames, 50 pixels
         Added cell 58 at 07, 105, 090, peak: 0.127, thresh: 5.000, 6002 frames, 53 pixels
         Added cell 59 at 01, 074, 057, peak: 0.126, thresh: 5.000, 5904 frames, 50 

/home/ali/packages/s2p-lbm/suite3d/extension.py:587: RuntimeWarning: invalid value encountered in divide
  frac_candidate_variance_on_roi = candidate_proj_on_roi / candidate_norms


         Added cell 1 at 14, 056, 237, peak: 0.204, thresh: 5.000, 6916 frames, 114 pixels
         Added cell 2 at 11, 037, 202, peak: 0.103, thresh: 5.000, 4837 frames, 25 pixels
         Added cell 3 at 09, 033, 272, peak: 0.057, thresh: 5.000, 4351 frames, 56 pixels
         Added cell 4 at 16, 058, 207, peak: 0.047, thresh: 5.000, 4650 frames, 17 pixels
         Added cell 5 at 16, 048, 272, peak: 0.043, thresh: 5.000, 4676 frames, 21 pixels
         Added cell 6 at 02, 028, 279, peak: 0.023, thresh: 5.000, 4916 frames, 9 pixels
         Added cell 7 at 02, 083, 274, peak: 0.023, thresh: 5.000, 4925 frames, 9 pixels
         Iter 0001: running 03 ROIs in parallel
         Added cell 8 at 10, 036, 204, peak: 0.075, thresh: 5.000, 4531 frames, 10 pixels
         Added cell 9 at 14, 066, 247, peak: 0.033, thresh: 5.000, 4767 frames, 11 pixels
         Added cell 10 at 15, 049, 275, peak: 0.022, thresh: 5.000, 4749 frames, 33 pixels
         Iter 0002: running 02 ROIs in parallel
    

...

         Added cell 113 at 07, 190, 031, peak: 0.217, thresh: 5.000, 6512 frames, 138 pixels
         Added cell 114 at 05, 144, 060, peak: 0.192, thresh: 5.000, 6645 frames, 141 pixels
         Added cell 115 at 07, 162, 092, peak: 0.192, thresh: 5.000, 5515 frames, 22 pixels
         Added cell 116 at 14, 124, 059, peak: 0.181, thresh: 5.000, 5940 frames, 56 pixels
         Added cell 117 at 04, 151, 079, peak: 0.178, thresh: 5.000, 6829 frames, 108 pixels
         Added cell 118 at 09, 143, 062, peak: 0.174, thresh: 5.000, 6074 frames, 131 pixels
         Added cell 119 at 07, 197, 085, peak: 0.173, thresh: 5.000, 7017 frames, 177 pixels
         Added cell 120 at 10, 163, 043, peak: 0.173, thresh: 5.000, 6861 frames, 86 pixels
         Added cell 121 at 07, 125, 072, peak: 0.172, thresh: 5.000, 6179 frames, 126 pixels
         Added cell 122 at 15, 185, 085, peak: 0.171, thresh: 5.000, 5837 frames, 46 pixels
         Added cell 123 at 17, 182, 064, peak: 0.170, thresh: 5.000, 5812 

/home/ali/packages/s2p-lbm/suite3d/extension.py:587: RuntimeWarning: invalid value encountered in divide
  frac_candidate_variance_on_roi = candidate_proj_on_roi / candidate_norms


         Added cell 161 at 07, 168, 051, peak: 0.144, thresh: 5.000, 7097 frames, 134 pixels
         Added cell 162 at 05, 197, 068, peak: 0.142, thresh: 5.000, 9806 frames, 214 pixels
         Added cell 163 at 03, 175, 068, peak: 0.142, thresh: 5.000, 6253 frames, 62 pixels
         Added cell 164 at 02, 118, 077, peak: 0.139, thresh: 5.000, 5669 frames, 16 pixels
         Added cell 165 at 16, 197, 082, peak: 0.137, thresh: 5.000, 6233 frames, 32 pixels
         Added cell 166 at 09, 193, 046, peak: 0.133, thresh: 5.000, 5616 frames, 60 pixels
         Added cell 167 at 09, 176, 025, peak: 0.133, thresh: 5.000, 5696 frames, 49 pixels
         Added cell 168 at 03, 154, 086, peak: 0.133, thresh: 5.000, 6231 frames, 43 pixels
         Added cell 169 at 02, 174, 091, peak: 0.131, thresh: 5.000, 6505 frames, 34 pixels
         Added cell 170 at 08, 159, 031, peak: 0.130, thresh: 5.000, 5727 frames, 76 pixels
         Added cell 171 at 13, 158, 017, peak: 0.129, thresh: 5.000, 4057 fram

/home/ali/packages/s2p-lbm/suite3d/extension.py:587: RuntimeWarning: invalid value encountered in divide
  frac_candidate_variance_on_roi = candidate_proj_on_roi / candidate_norms
/home/ali/packages/s2p-lbm/suite3d/extension.py:587: RuntimeWarning: invalid value encountered in divide
  frac_candidate_variance_on_roi = candidate_proj_on_roi / candidate_norms


         Added cell 81 at 09, 213, 053, peak: 0.183, thresh: 5.000, 6672 frames, 146 pixels
         Added cell 82 at 10, 230, 037, peak: 0.173, thresh: 5.000, 6561 frames, 132 pixels
         Added cell 83 at 16, 215, 027, peak: 0.158, thresh: 5.000, 6452 frames, 51 pixels
         Added cell 84 at 03, 263, 088, peak: 0.155, thresh: 5.000, 7058 frames, 119 pixels
         Added cell 85 at 03, 227, 044, peak: 0.152, thresh: 5.000, 469 frames, 652 pixels
         Added cell 86 at 14, 233, 020, peak: 0.148, thresh: 5.000, 5732 frames, 87 pixels
         Added cell 87 at 05, 272, 057, peak: 0.146, thresh: 5.000, 5831 frames, 138 pixels
         Added cell 88 at 14, 218, 075, peak: 0.146, thresh: 5.000, 6029 frames, 57 pixels
         Added cell 89 at 06, 221, 039, peak: 0.146, thresh: 5.000, 6068 frames, 103 pixels
         Added cell 90 at 03, 246, 061, peak: 0.145, thresh: 5.000, 5971 frames, 91 pixels
         Added cell 91 at 13, 216, 014, peak: 0.145, thresh: 5.000, 4093 frames, 600 

In [ ]:
# rois_dir_path = job.combine_patches(n.arange(80), job.dirs['rois'], deduplicate=False,
#                                     parent_dir_name='segmentation', info_use_idx=None)

In [ ]:
job.compute_npil_masks(stats_dir = job.dirs['rois'])


In [ ]:

traces = job.extract_and_deconvolve(stats_dir=job.dirs['rois'])

   Updated main params file
   Movie shape: (24, 53470, 1044, 737)
103187
   Extracting 103187 valid cells, and saving cell flags to /mnt/md0/runs/s3d-SS002_2024-08-12_1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-17-18-19-20-21/rois/iscell_extracted.npy
   Extracting activity
         Will extract in 107 batches of 500
   Saving intermediate results to /mnt/md0/runs/s3d-SS002_2024-08-12_1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-17-18-19-20-21/rois


In [6]:
job.export_results('/mnt/zeytin-f1/s3d-results/',result_dir_name='rois')

   Created dir /mnt/zeytin-f1/s3d-results/s3d-results-SS002_2024-08-12_1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-17-18-19-20-21 to export results
      Loading from /mnt/md0/runs/s3d-SS002_2024-08-12_1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-17-18-19-20-21/rois/stats_small.npy
      Loading from /mnt/md0/runs/s3d-SS002_2024-08-12_1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-17-18-19-20-21/rois/info.npy
      Loading from /mnt/md0/runs/s3d-SS002_2024-08-12_1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-17-18-19-20-21/rois/F.npy
      Loading from /mnt/md0/runs/s3d-SS002_2024-08-12_1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-17-18-19-20-21/rois/spks.npy
      Loading from /mnt/md0/runs/s3d-SS002_2024-08-12_1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-17-18-19-20-21/rois/Fneu.npy
      Loading from /mnt/md0/runs/s3d-SS002_2024-08-12_1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-17-18-19-20-21/rois/iscell.npy
   /mnt/zortex-subjects/SS002/2024-08-12/2/2024-08-12_2_SS002_2P_00001_00001.tif is 100 frames and 3971882544 bytes
   